In [2]:
## TASKS :
# LAST PRINT IN THE MAIN FILE

In [1]:
## LIBRARIES ##

import requests as rq
import bs4
import re

import pandas as pd
import numpy as np

import datetime as dt

In [2]:
## DOWNLADING FILE LOCATION ##

download_path = r'C:\Users\Atlantis\Documents\Jupyter\Meteo'
synop_file_name = 'SYNOP_stations.txt'

In [37]:
## MAIN FILE ##
# Function file must be run beforehand
# This program works in theory for all countries provided that the urls and the data handling are changed
# but be careful of relative time with large countries like Russia.

# Dowloading and reading location of each weather station given by postesSynop.csv and making a dataframe from it
df_stations = stations_init(download_path,synop_file_name) 

# Asking for the position of the city (must be in mainland France) 

#city_lat = float(input('Give the latitude of the city: '))
#city_long = float(input('Give the longitude of the city: '))

# example: Juan-les-Pins 
city_lat = 43.5
city_long = 7.1


# Finding the nearest weather station for a city and giving its ID
closest = min(df_stations.iterrows(), key = lambda df: (df[1][0]-city_lat)**2 + (df[1][1]-city_long)**2)
stationID = closest[0]
#print(closestID)

# example: Juan-les-Pins
# closestID = 7690

# Asking the date of the report (must be within 15 days)
#time, day, month, year = [int(x) for x in input('Give the time, date, month and year of the desired report').split()]

# example : Juan-les-Pins
time, day, month, year = [4, 21, 1, 2021]

chosen_date = dt.date(year,month,day)

while check_date(chosen_date) == 0:
    time, day, month, year = [int(x) for x in input('Give the time, date, month and year of the desired report').split()]
    chosen_date = dt.date(year,month,day)


# Finding the report for the chosen date
df_data = report(chosen_date,time)

# Finding the report of the nearest station and printing it
#print_report(stationID,df_data)

The given date is available in the database.
The closest hourly report from the time given is from the 21 January 2021 at 3.
['numer_sta', 'date', 'pmer', 'tend', 'cod_tend', 'dd', 'ff', 't', 'td', 'u', 'vv', 'ww', 'w1', 'w2', 'n', 'nbas', 'hbas', 'cl', 'cm', 'ch', 'pres', 'niv_bar', 'geop', 'tend24', 'tn12', 'tn24', 'tx12', 'tx24', 'tminsol', 'sw', 'tw', 'raf10', 'rafper', 'per', 'etat_sol', 'ht_neige', 'ssfrai', 'perssfrai', 'rr1', 'rr3', 'rr6', 'rr12', 'rr24', 'phenspe1', 'phenspe2', 'phenspe3', 'phenspe4', 'nnuage1', 'ctype1', 'hnuage1', 'nnuage2', 'ctype2', 'hnuage2', 'nnuage3', 'ctype3', 'hnuage3', 'nnuage4', 'ctype4', 'hnuage4']


In [47]:
## FUNCTION FILE

def stations_init(download_path,synop_file_name):
## Dowloading and reading location of each weather station given by postesSynop.csv and making a dataframe from it

# Inputs :
# dowload_path (str)
# synop_file_name (str)
# Ouput :
# df_stations (dataframe)

    # Checking if the file is already dowloaded
    try :
        with open(download_path + '\ ' + synop_file_name, 'x') as csv_file: # two versions because it didn't work with mode 'w+' and mode 'x' do not recreate a file
            url = 'https://donneespubliques.meteofrance.fr/donnees_libres/Txt/Synop/postesSynop.csv'
            response = rq.get(url) # search the data if the file does not exist
        
            csv_file.write(response.text) # write the data on the file if the file does not exist
    
        with open(download_path + '\ ' + synop_file_name, 'r') as csv_file: # read the file
            df_stations = pd.read_csv(csv_file, delimiter = ';')
            df_stations = df_stations.drop(columns =['Nom','Altitude'])
            df_stations = df_stations.set_index('ID')
            #df_stations = df_stations.apply(pd.to_numeric) # already applied by read_csv
    
    except FileExistsError :
    
        with open(download_path + '\ ' + synop_file_name, 'r') as csv_file: # only read the file if the file exists
            df_stations = pd.read_csv(csv_file, delimiter = ';')
            df_stations = df_stations.drop(columns =['Nom','Altitude'])
            df_stations = df_stations.set_index('ID')
            #df_stations = df_stations.apply(pd.to_numeric) # already applied by read_csv
    return df_stations


def check_date(chosen_date):
## Checking if the selected date is in the range of 2 weeks

# Input :
# chosen_date (date.py object)
# Output :
# 0 if the chosen_date is not within 15 days from today 1 if it is (boolean)

    delta_day = (pd.to_datetime('today').date() - chosen_date).days
    if (delta_day > 14) or (delta_day< 1):
        print('Not a good date.')
        return 0
    else:
        print('The given date is available in the database.')
        return 1

    
    

def report(chosen_date,time,value_asked = ['t','u']):
## Giving the asked data from the weather report from the chosen day at the chosen time, by default temperature and humidity

# Input : 
# chosen_date (date.py object)
# time (int)
# value_asked (list of str even if there is only one str)
# Output :
# df_data (dataframe)
    
    index_timelist = np.argmin(abs(np.array([*range(0,25,3)]) - time))
    
    # Checking if the selected time is in range of 3h if not promting a message saying 'the hour the nearest was ...'
    if (index_timelist == 8):
        chosen_date = chosen_date + dt.timedelta(days=1)
        time = 0 # change the date to the beginning of the next day
    else:
        time = index_timelist*3 # round to the time of the actual available report
        
    print('The closest hourly report from the time given is from the {} at {}.'.format(chosen_date.strftime('%d %B %Y'),time))

    
    # Downloading the data from the URL
    # IMPROVEMENT: make a double grab with search of csv file instead of the exact url
    url = 'https://donneespubliques.meteofrance.fr/donnees_libres/Txt/Synop/synop.{}.csv'.format(chosen_date.strftime('%Y%m%d')+'{:02d}'.format(time))
    #print(url)
    response = rq.get(url) 
    #print(response.text)
    
    line_list = response.text.split('\n') # splitline() doesn't work 
    line_list = list(filter(None,line_list)) # remove eventual empty elements
    #print(line_list)
    
    columns = line_list[0].split(';') # create a list from the string of separated elements of the csv file
    columns = list(filter(None, columns)) # remove eventual empty elements
    line_list.pop(0) # remove the columns' names from the line_list
    #print(columns)
    
    data =[]

    for line in line_list: # always better to create a dataframe from list at the last moment
        a = line.split(';') # create a list from the string of separated elements of the csv file
        a = list(filter(None, a)) # remove eventual empty elements
        data.append(a)

    df_data = pd.DataFrame(data, columns = columns)
    df_data = df_data.replace('mq', np.nan) # replace 'mq' with Nan
    df_data =df_data.set_index(columns[0])
    #df_data.head(5)
    
    # Delivering only the columns of the asked data
    df_data = df_data[value_asked] # t is the temperature in K and u is the humidity in % 
                                   # for the list of the other parameters see annexes
    return df_data


def print_report(stationID, df_data):
## Gives the report for the given station
    
# Input :
#
#
    
    bool_list = [int(x)==stationID for x in df_data.index] 

    rea = df_data[bool_list]# make a case for complete a false list # convert to a dict?
    return rea



In [14]:
## ANNEXES

# SYNOP stations list
# https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=90&id_rubrique=32

# Weather observations explanation
# https://donneespubliques.meteofrance.fr/?fond=donnee_libre&prefixe=Txt%2FSynop%2Fsynop&extension=csv&date=20201211&reseau=12 20201211 and 12 can be changed 

In [58]:
## TEST WINDOW
x=[int(x)==stationID for x in df_data.index] 
print(df_data[x])

                    t   u
numer_sta                
07690      281.550000  92
